In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

class Camera():
    
    def __init__(self):
        self.nx = 9
        self.ny = 6
        self.cal_points()
        
    def cal_points(self):
        self.objpoints = []
        self.imgpoints = []
        
        objp = np.zeros((self.nx * self.ny, 3), np.float32)
        objp[:, :2] = np.mgrid[0 : self.nx, 0 : self.ny].T.reshape(-1, 2)

        imgs = glob.glob('camera_cal/calibration*.jpg')
        for fn in imgs:
            # Read each chessboard image
            img = cv2.imread(fn)
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (self.nx, self.ny), None)

            # If found, draw corners
            if ret == True:
                self.imgpoints.append(corners)
                self.objpoints.append(objp)

    def undist(self, img):
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera( \
                                    self.objpoints, self.imgpoints, img.shape[1::-1], None, None)
        return cv2.undistort(img, mtx, dist, None, mtx)

class Lane():
    
    def __init__(self):
        self.camera = Camera()
        self.initTransform()
        self.left_fit = None
        self.right_fit = None
        self.detected = False
    
    def initTransform(self):
        src = np.float32([ [205, 719], [594, 450], [686, 450], [1105, 719] ])
        dst = np.float32([ [200, 719], [200, 0], [900, 0], [900, 719] ])
        self.trans_M = cv2.getPerspectiveTransform(src, dst)
        self.trans_Minv = cv2.getPerspectiveTransform(dst, src)
    
    def sobel_xy(self, img, orient='x', kernel=3, thresh=(20, 100)):
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        if orient == 'x':
            sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel))
        if orient == 'y':
            sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel))
        # Rescale back to 8 bit integer
        sobel = np.uint8(255*sobel/np.max(sobel))
        # Create a copy and apply the threshold
        binary = np.zeros_like(sobel)
        binary[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1
        return binary
    
    def R_channel(self, img, thresh = (200, 255)):
        R = img[:,:,0]
        binary = np.zeros_like(R)
        binary[(R > thresh[0]) & (R <= thresh[1])] = 1
        return binary

    """Should be uninstorted image"""
    def combine(self, undist_img):
        sobelx = self.sobel_xy(undist_img, thresh=(10, 100))
        #S = s_channel(undist_img, thresh=(200, 255))
        R = self.R_channel(undist_img, thresh=(210, 255))

        # R-channel +　sobel-x
        com = np.zeros_like(R)
        com[(R == 1) | (sobelx == 1)] = 1

        return com

    def warp(self, com_img):
        img_size = (com_img.shape[1], com_img.shape[0])
        return cv2.warpPerspective(com_img, self.trans_M, img_size, flags=cv2.INTER_LINEAR)
    
    def unwarp(self, img):
        img_size = (img.shape[1], img.shape[0])
        return cv2.warpPerspective(img, self.trans_Minv, img_size, flags=cv2.INTER_LINEAR)
    
    """Find lane from scratch"""
    def __find_lane_0(self, warped):
        hist = np.sum(warped[int(warped.shape[0]/2):,:], axis=0)
        midpoint = np.int(hist.shape[0]/2)
        leftx_base = np.argmax(hist[:midpoint])
        rightx_base = np.argmax(hist[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = warped.shape[0] - (window+1)*window_height
            win_y_high = warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & \
                              (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & \
                               (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        return left_fit, right_fit
    
    """Find lane from exist lane"""
    def __find_lane_1(self, warped):
        nonzero = warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        A, B, C = self.left_fit
        left_lane_inds = ((nonzerox > (A * (nonzeroy**2) + B * nonzeroy + C - margin)) \
                          & (nonzerox < (A * (nonzeroy**2) + B * nonzeroy + C + margin)))
        A, B, C = self.right_fit
        right_lane_inds = ((nonzerox > (A * (nonzeroy**2) + B * nonzeroy + C - margin)) \
                            & (nonzerox < (A * (nonzeroy**2) + B * nonzeroy + C + margin)))

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        return left_fit, right_fit

    def find_lane(self, warped):
        if self.detected:
            self.left_fit, self.right_fit = self.__find_lane_1(warped)
        else:
            self.left_fit, self.right_fit = self.__find_lane_0(warped)
            self.detected = True
        # Generate x and y values for plotting
        ploty = np.linspace(0, warped.shape[0] - 1, warped.shape[0] )
        A, B, C = self.left_fit
        left_fitx = A * ploty**2 + B * ploty + C
        A, B, C = self.right_fit
        right_fitx = A * ploty**2 + B * ploty + C
        
        return left_fitx, right_fitx, ploty
        
    def draw_lane(self, img):
        undist_img = self.camera.undist(img)

        com_img = self.combine(undist_img)

        img_size = (img.shape[1], img.shape[0])
        
        warped = self.warp(com_img)

        left_fitx, right_fitx, ploty = self.find_lane(warped)

        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, self.trans_Minv, (undist_img.shape[1], undist_img.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(undist_img, 1, newwarp, 0.3, 0)
        self.draw_text(result)

        return result
    
    def draw_text(self, img):
        ploty = np.linspace(0, img.shape[0] - 1, img.shape[0] )
        A, B, C = self.left_fit
        leftx = A * ploty**2 + B * ploty + C
        A, B, C = self.right_fit
        rightx = A * ploty**2 + B * ploty + C
        
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension

        y = img.shape[0]
        d = cv2.perspectiveTransform(np.float32([[ [leftx[-1], y], [rightx[-1], y] ]]), self.trans_Minv)
        offset = ((d[0][0][0] + d[0][1][0]) / 2 - img.shape[1] / 2) * xm_per_pix
        y = y * ym_per_pix
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)
        # Calculate the new radii of curvature
        A, B, C = left_fit_cr
        left_rad = ((1 + (2 * A * y + B)**2)**1.5) / np.absolute(2 * A)
        A, B, C = right_fit_cr
        right_rad = ((1 + (2 * A * y + B)**2)**1.5) / np.absolute(2 * A)
        
        cv2.putText(img, 'Radius of Curvature: {:>4.0f}(m), {:>4.0f}(m)'.format(left_rad, right_rad), (50, 60), \
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=3)
        cv2.putText(img, 'Vehicle offset of center: {:.2f}m'.format(offset), (50, 120), \
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=3)

In [2]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

lane = Lane()

video_output = 'output.mp4'
clip = VideoFileClip('project_video.mp4')

output_clip = clip.fl_image(lane.draw_lane)
%time output_clip.write_videofile(video_output, audio=False)


[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|█████████▉| 1260/1261 [11:20<00:00,  1.79it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 11min 7s, sys: 28.7 s, total: 11min 36s
Wall time: 11min 20s
